In [2]:
import itertools

In [3]:
mentee_prefs = {'m1':['w1','w5','w4','w3','w2'],
                'm2':['w1','w3','w5','w2','w4'],
                'm3':['w4','w2','w3','w5','w1'],
                'm4':['w5','w3','w2','w1','w4']}

worker_prefs = {'w1':['m4','m2','m1','m3'],
                'w2':['m1','m2','m3','m4'],
                'w3':['m1','m3','m4','m2'],
                'w4':['m4','m2','m1','m3'],
                'w5':['m4','m2','m3','m1']}

doctor_prefs = {'d1':[['h3','h1','h2'],1],
                'd2':[['h2','h1','h3'],1],
                'd3':[['h1','h3','h2'],1],
                'd4':[['h1','h2','h3'],1]}

hospital_prefs = {'h1':[['d1','d2','d3','d4'],2],
                  'h2':[['d1','d2','d3','d4'],1],
                  'h3':[['d3','d1','d2','d4'],1]}

hospital_prefs_alt = {'h1':[['d2','d1','d3','d4'],2],
                      'h2':[['d1','d2','d3','d4'],1],
                      'h3':[['d3','d1','d2','d4'],1]}

midterm_2024_q1_students = {'i1':['s1','s2'],
                            'i2':['s1','s2'],
                            'i3':['s2','s1']}
midterm_2024_q1_schools = {'s1':['i1','i2','i3'],
                           's2':['i2','i1','i3'],
                           's3':['i1','i2','i3']}

Matching Algorithms

In [ ]:
def matching(proposer_prefs: dict, receiver_prefs: dict):

    current_proposer_matches = {}
    current_proposer_non_matches = [[i,0] for i in proposer_prefs.keys()]
    lonely_hearts = []

    leftovers = len(proposer_prefs) - len(receiver_prefs)
    if leftovers<0:
        leftovers = 0

    while (len(current_proposer_matches) + len(lonely_hearts)) != len(proposer_prefs) :

        j = current_proposer_non_matches[0]
        prop, highest_prop = j      #assigns variables for the current proposer and their next highest proposal

        if highest_prop <= len(proposer_prefs[prop])-1:
            receiver = proposer_prefs[prop][highest_prop]       #finds next highest receiver that they haven't proposed to
            receiver_pref_pos = receiver_prefs[receiver].index(prop)      #finds that proposer's pos in receiver's prefs
        
            if receiver not in current_proposer_matches.keys():      #if the current receiver hasn't been previously matched then:

                current_proposer_matches[receiver] = [prop, receiver_pref_pos, highest_prop]        #an entry for the current receiver is added
                current_proposer_non_matches.remove(j)      #the proposer's current entry in the non-matches list is removed

            else:           #if current receiver has been matched:
                
                if current_proposer_matches[receiver][1] > receiver_prefs[receiver].index(prop):        #if the receiver prefers challenger to current match:

                    current_proposer_non_matches.remove(j)      #that challenger is removed from non matches
                    current_proposer_non_matches.append([current_proposer_matches[receiver][0],current_proposer_matches[receiver][2]+1])     #the previous match is added to the non-matches
                    current_proposer_matches[receiver] = [prop, receiver_pref_pos, highest_prop]        #the receiver's entry in the matches dict is updated to add the new match

                else:       #if the receiver doesn't prefer the current challenger, that challenger moves to the next proposal in their list
                    
                    current_proposer_non_matches.remove(j)
                    current_proposer_non_matches.append([prop,highest_prop+1])

        else:
            lonely_hearts.append(prop)
    
    return f"Matches: {current_proposer_matches}. Lonely Hearts: {lonely_hearts}"

def multiple_matching(proposer_prefs: dict, receiver_prefs: dict):

    current_proposer_matches = {i:[] for i in receiver_prefs.keys()}
    current_proposer_non_matches = [i for i in proposer_prefs.keys()]

    leftovers = sum([i[1] for i in proposer_prefs.values()]) - sum([i[1] for i in receiver_prefs.values()])
    if leftovers<0:
        leftovers = 0

    prop_caps = {i:j[1] for i,j in proposer_prefs.items()}
    rec_caps = {i:j[1] for i,j in receiver_prefs.items()}

    prop_current_prefs = {i:0 for i in proposer_prefs.keys()}
    rec_current_prefs = {i:0 for i in receiver_prefs.keys()}

    while len(current_proposer_non_matches) != leftovers:

        prop = current_proposer_non_matches[0]
        highest_prop = prop_current_prefs[prop]
        # print(j)
        
        receiver = proposer_prefs[prop][0][highest_prop]       #finds next highest receiver that they haven't proposed to
        receiver_pref_pos = receiver_prefs[receiver][0].index(prop)      #finds that proposer's pos in receiver's prefs
        
        # print(current_proposer_matches[receiver][0])

        if rec_caps[receiver]>0:      #if the current receiver has capacity then:

            current_proposer_matches[receiver].append([prop, rec_current_prefs[receiver],prop_current_prefs[prop]])        #an entry for the current receiver is added
            current_proposer_matches[receiver] = sorted(current_proposer_matches[receiver], key = lambda x: x[1])
            # current_proposer_matches[receiver][0] -= 1
            rec_caps[receiver] -= 1

            if prop_caps[prop] == 1:    
                current_proposer_non_matches.remove(prop)      #the proposer's current entry in the non-matches list is removed if at capacity
            else:
                prop_caps[prop] -= 1
                current_proposer_non_matches.remove(prop)
                current_proposer_non_matches.append(prop)

        else:           #if current receiver has been matched and is at capacity:
            # print(current_proposer_matches[receiver][-1][0],receiver_prefs[receiver][0].index(prop))
            if current_proposer_matches[receiver][-1][1] > receiver_prefs[receiver][0].index(prop):        #if the receiver prefers challenger to current least preferred match:

                if prop_caps[prop] == 1:    
                    current_proposer_non_matches.remove(prop)      #the proposer's current entry in the non-matches list is removed if he's got no more capacity
                else:
                    prop_caps[prop] -= 1
                    current_proposer_non_matches.remove(prop)
                    current_proposer_non_matches.append(prop)

                current_proposer_non_matches.append(current_proposer_matches[receiver][-1][0])     #the previous match is added to the non-matches
                prop_current_prefs[prop] += 1
                prop_caps[current_proposer_matches[receiver][-1][0]] += 1

                current_proposer_matches[receiver].append([prop, rec_current_prefs[receiver],prop_current_prefs[prop]])        #an entry for the current receiver is added
                current_proposer_matches[receiver] = sorted(current_proposer_matches[receiver], key = lambda x: x[1])              
                del current_proposer_matches[receiver][-1]

            else:       #if the receiver doesn't prefer the current challenger, that challenger moves to the next proposal in their list
                
                current_proposer_non_matches.remove(prop)
                current_proposer_non_matches.append(prop)
                prop_current_prefs[prop] += 1
        
        # print(current_proposer_matches,current_proposer_non_matches, prop, prop_caps[prop])
    
    return current_proposer_matches
        

# Assignment Algorithms

In [5]:
def serial_dict(indiv_prefs : dict, objects: list, indiv_ordering: list):

    objects_1 = objects
    indiv_prefs_1 = indiv_prefs
    indiv_ordering_1 = indiv_ordering
    result = {}

    # print(objects)

    while indiv_ordering_1:
        i = indiv_ordering_1[0]
        counter = 0
        while i not in result.keys():
            i_pref = indiv_prefs_1[i][counter]
            if i_pref in objects_1:
                result[i] = i_pref
                indiv_ordering_1.remove(i)
                objects_1.remove(i_pref)
            counter+=1
    
    return result

def top_trading_cycle(indiv_prefs : dict, endowments: dict):

    results = {}

    while endowments:

        current_prefs = {i:indiv_prefs[i][0] for i in indiv_prefs.keys()}
        endow_inverted = {endowments[i]:i for i in endowments.keys()}

        matrix = []
        starter = list(current_prefs.keys())[0]

        while matrix.count(starter)<=1:
            matrix.append(starter)
            starter = endow_inverted[current_prefs[starter]]

        cycle = matrix[matrix.index(matrix[-1]):-1]
        print(matrix,cycle)

        for i in cycle:
            results[i] = current_prefs[i]
            endowments.pop(i)
            indiv_prefs.pop(i)
            removals = [current_prefs[k] for k in cycle]
            for j in indiv_prefs.keys():
                for l in removals:
                    if l in indiv_prefs[j]:
                        indiv_prefs[j].remove(l)
        
    return results

Serial Dict Basic

In [14]:
tenants = {'i1':['h3','h5','h1','h4','h2'],
           'i2':['h5','h1','h3','h2','h4'],
           'i3':['h3','h4','h2','h1','h5'],
           'i4':['h5','h1','h2','h3','h4'],
           'i5':['h4','h2','h1','h5','h3']}

houses = ['h1','h2','h3','h4','h5']
house_endowments = {'i1':'h1','i2':'h2','i3':'h3','i4':'h4','i5':'h5'}

tenant_order = ['i1','i2','i3','i4','i5']

hypothetical_tenant_orders = list(itertools.permutations(tenant_order))
for i in hypothetical_tenant_orders:
    hypothetical_tenant_orders[hypothetical_tenant_orders.index(i)] = list(i)

target = {'i1':'h1','i2':'h2','i3':'h3','i4':'h5','i5':'h4'}
count=0

while serial_dict(tenants, houses, hypothetical_tenant_orders[count]) != target:
    houses = ['h1','h2','h3','h4','h5']
    # print(count)
    count+=1

print(hypothetical_tenant_orders[count])
# print(serial_dict(tenants, houses, tenant_order))
# print(top_trading_cycle(tenants, house_endowments))

[]


Mentee Optimal Matching

In [26]:
print(matching(midterm_2024_q1_students,midterm_2024_q1_schools))

i1 0 ['s1', 's2']
i2 0 ['s1', 's2']
i3 0 ['s2', 's1']
i2 1 ['s1', 's2']
i3 1 ['s2', 's1']
i3 2 ['s2', 's1']
Matches: {'s1': ['i1', 0, 0], 's2': ['i2', 0, 1]}. Lonely Hearts: ['i3']


Worker Optimal Matching

In [ ]:
print(matching(worker_prefs,mentee_prefs))

{'m4': ['w5', 0, 0], 'm1': ['w4', 2, 2], 'm2': ['w1', 0, 1], 'm3': ['w2', 1, 2]}


Doctor Optimal Matching

In [ ]:
print(multiple_matching(doctor_prefs,hospital_prefs))

{'h1': [['d3', 2], ['d4', 3]], 'h2': [['d2', 1]], 'h3': [['d1', 1]]}


Hospital Optimal Matching

In [ ]:
print(multiple_matching(hospital_prefs_alt,doctor_prefs))

{'d1': [['h2', 0, 0]], 'd2': [['h1', 0, 0]], 'd3': [['h3', 0, 0]], 'd4': [['h1', 0, 3]]}
